<h2> 1. Data preparation </h2>

In [8]:
# Importing important libraries
import numpy as np
import os 
import shutil
import pickle
import cv2
import time

<h4> 1.1 Extracting frames from video </h4>

In [12]:
def video_to_frames(input_loc, output_loc):
    """Function to extract frames from input video file (any format : .mp4, .avi, .MTS)
    and save them as separate frames in an output directory.
    Args:
        input_loc: Input video file.
        output_loc: Output directory to save the frames.
    Returns:
        None
    """
    try:
        os.mkdir(output_loc)
    except OSError:
        pass
    # Start capturing the feed
    cap = cv2.VideoCapture(input_loc)
    # Find the number of frames
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    # Will capture 50 frames from the whole video
    frame_threshold = 50
    frame_count = 0
    # Start converting the video
    while cap.isOpened():
        # Extract the frame
        ret, frame = cap.read()
        # Write the results back to output location.
        cv2.imwrite(output_loc + "\\%#05d.jpg" % (frame_count+1), frame)
        frame_count += 1
        # If frame threshold is reached stop extraction
        if (frame_count > (frame_threshold-1)):
            # Release the feed
            cap.release()
            break
    return

In [20]:
srcPath = "C:\\Users\\smitr\\Downloads\\UCF-10\\"
dst_path = r"C:\Users\smitr\Desktop\extracted_frames"
os.mkdir(dst_path)

In [21]:
classes = os.listdir(srcPath)
dst_dir = dst_path + "\\"
start = time.time()

for c in classes:
    files = os.listdir(srcPath+c)
    os.mkdir(dst_dir + c)
    for f in files:
        # Video filepath
        filename = srcPath + c + '\\' + f 
        # Filepath of images to be extracted
        dst_path  = dst_dir + c + '\\' + f[:-4]
        video_to_frames(filename, dst_path)
    print("Time taken for completing class {} is {} secs".format(c, time.time()-start))
        
print("Time taken for completion: {} secs".format(time.time()-start))

Time taken for completing class ApplyEyeMakeup is 82.35307931900024 secs
Time taken for completing class ApplyLipstick is 146.15206480026245 secs
Time taken for completing class Archery is 219.55078864097595 secs
Time taken for completing class BabyCrawling is 273.1600503921509 secs
Time taken for completing class BalanceBeam is 317.40345215797424 secs
Time taken for completing class BandMarching is 408.73483395576477 secs
Time taken for completing class BaseballPitch is 473.7194724082947 secs
Time taken for completing class Basketball is 580.9868850708008 secs
Time taken for completing class BasketballDunk is 697.4812254905701 secs
Time taken for completing class BenchPress is 822.8519630432129 secs
Time taken for completion: 822.8519630432129 secs


<h4> 1.2 Train test split </h4>

In [24]:
trainList = []
testList  = []
path = r"C:\Users\smitr\Desktop\extracted_frames"

classes = os.listdir(path)
for c in classes:
    trainIndvList = []
    vidList = os.listdir(path + "\\" + c)
    vidList.sort()
    for item in vidList:
        # video name eg: v_ApplyEyeMakeup_g01_c01 => g01
        user = item.split('_')[2]
        if (user not in trainIndvList):
            if len(trainIndvList) < 20:
                trainIndvList.append(user) # Keeping track of train-test list
                trainList.append(item) # Adding the video name to train list
            else:
                testList.append(item) # Adding the video name to test list
        else:
            trainList.append(item)

In [27]:
with open('trainList_10class.pckl','wb') as f:
    pickle.dump(trainList,f)
with open('testList_10class.pckl','wb') as f:
    pickle.dump(testList,f)

In [33]:
# Filtering train set
count = 1
trainDelList = []
for item in trainList:
    cl = item.split('_')[1]
    src_path = path + "\\" + cl + "\\" + item
    fNames = os.listdir(src_path)
    fNums = [int(x[:-4].split('_')[-1]) for x in fnames]
    if fNums[-1]-len(fNames)>1:
        trainDelList.append(path + "\\" + cl + '\\' + item)
    count += 1    
for item in trainDelList:
    shutil.rmtree(item)

In [34]:
# Filtering test set
count = 1
testDelList = []
for item in testList:
    cl = item.split('_')[1]
    src_path = path + "\\" + cl + "\\" + item
    fNames = os.listdir(src_path)
    fNums = [int(x[:-4].split('_')[-1]) for x in fnames]
    if fNums[-1]-len(fNames)>1:
        testDelList.append(path + "\\" + cl + '\\' + item)
    count += 1    
for item in testDelList:
    shutil.rmtree(item)

In [35]:
classes

['ApplyEyeMakeup',
 'ApplyLipstick',
 'Archery',
 'BabyCrawling',
 'BalanceBeam',
 'BandMarching',
 'BaseballPitch',
 'Basketball',
 'BasketballDunk',
 'BenchPress']

In [36]:
train_dir = r"C:\Users\smitr\Desktop\train_10class"
os.mkdir(train_dir)
print("Created training directory")

test_dir = r"C:\Users\smitr\Desktop\test_10class"
os.mkdir(test_dir)
print("Created testing directory")

Created training directory
Created testing directory


In [37]:
path = r"C:\Users\smitr\Desktop\extracted_frames"

for class_ in classes:
    src_path = path + "\\" + class_
    files = os.listdir(src_path)
    train_num = np.floor(len(files)*0.8)
    test_num = len(files) - train_num
    
    for idx in range(int(train_num)):
        train_dst = train_dir + "\\" + class_ + "\\" + files[idx]
        shutil.copytree(src_path+"\\"+files[idx], train_dst)

    for idx2 in range(int(train_num),int(train_num+test_num)):
        test_dst = test_dir + "\\" + class_ + "\\" + files[idx2]
        shutil.copytree(src_path+"\\"+files[idx2], test_dst)